In [39]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,6 hours 37 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,"7 days, 10 hours and 44 minutes"
H2O_cluster_name:,H2O_from_python_kevin_0sjooy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.485 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [40]:
train = pd.read_csv('input/train.csv', index_col=0)
pred = pd.read_csv('input/predict.csv', index_col=0)

In [41]:
h2train = h2o.H2OFrame(train)
h2pred = h2o.H2OFrame(pred)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [42]:
x = list(train.drop(columns = 'price').columns)
y = 'price'

In [15]:
automl = H2OAutoML(max_models = 50, seed = 1, max_runtime_secs = 1200, sort_metric = 'RMSE')
automl.train(x,y,training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [17]:
leader_board = automl.leaderboard
leader_board.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201005_194209,529.388,280252,280252,266.561,0.0951778
StackedEnsemble_BestOfFamily_AutoML_20201005_194209,532.304,283348,283348,265.926,0.0914547
GBM_4_AutoML_20201005_194209,534.884,286101,286101,268.69,0.0925132
GBM_3_AutoML_20201005_194209,537.393,288791,288791,274.868,nan
GBM_2_AutoML_20201005_194209,537.944,289384,289384,277.682,nan
GBM_1_AutoML_20201005_194209,538.122,289575,289575,279.254,0.102121
GBM_grid__1_AutoML_20201005_194209_model_9,542.849,294685,294685,275.191,0.0980034
GBM_grid__1_AutoML_20201005_194209_model_8,544.368,296336,296336,270.737,0.0916348
GBM_grid__1_AutoML_20201005_194209_model_6,547.381,299626,299626,273.763,nan
GBM_5_AutoML_20201005_194209,558.553,311982,311982,277.542,0.0967154


In [28]:
predictions = automl.leader.predict(h2pred)
predictions.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
756.893
6038.14
5882.9
1089.92
3196.89
550.888
1432.67
872.304
8424.65
697.669


In [33]:
pgr = predictions.as_data_frame()
columns=['id','price']
pgr = pgr.reset_index(level=0)
pgr.columns = columns
pgr.to_csv('last_try.csv', index = False)

In [47]:
automl = H2OAutoML(max_models = 50, seed = 1, max_runtime_secs = 2400, sort_metric = 'RMSE')
automl.train(x,y,training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [48]:
leader_board2 = automl.leaderboard
leader_board2.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_030128,529.631,280509,280509,266.478,0.0947449
StackedEnsemble_BestOfFamily_AutoML_20201006_030128,532.36,283408,283408,265.758,0.0910183
GBM_4_AutoML_20201006_030128,534.884,286101,286101,268.69,0.0925132
GBM_3_AutoML_20201006_030128,537.393,288791,288791,274.868,nan
GBM_2_AutoML_20201006_030128,537.944,289384,289384,277.682,nan
GBM_1_AutoML_20201006_030128,538.122,289575,289575,279.254,0.102121
GBM_grid__1_AutoML_20201006_030128_model_12,542.849,294685,294685,275.191,0.0980034
GBM_grid__1_AutoML_20201006_030128_model_11,544.368,296336,296336,270.737,0.0916348
GBM_grid__1_AutoML_20201006_030128_model_9,547.381,299626,299626,273.763,nan
GBM_5_AutoML_20201006_030128,553.041,305854,305854,275.314,0.0967703


In [49]:
predictions2 = automl.leader.predict(h2pred)
predictions2.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
755.76
6042.24
5881.47
1090.64
3195.83
550.223
1430.6
872.462
8432.67
696.199


In [50]:
pgr2 = predictions2.as_data_frame()
columns=['id','price']
pgr2 = pgr2.reset_index(level=0)
pgr2.columns = columns
pgr2.to_csv('last_try_2.csv', index = False)